In [181]:
from parser_transitions import *
from parser_model import *
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [182]:

def test_dependencies(name, deps, ex_deps):
    """Tests the provided dependencies match the expected dependencies"""
    deps = tuple(sorted(deps))
    assert deps == ex_deps, \
        "{:} test resulted in dependency list {:}, expected {:}".format(name, deps, ex_deps)


def test_minibatch_parse():
    """Simple tests for the minibatch_parse function
    Warning: these are not exhaustive
    """

    # Unidirectional arcs test
    sentences = [["right", "arcs", "only"],
                 ["right", "arcs", "only", "again"],
                 ["left", "arcs", "only"],
                 ["left", "arcs", "only", "again"]]
    deps = minibatch_parse(sentences, DummyModel(), 2)
    test_dependencies("minibatch_parse", deps[0],
                      (('ROOT', 'right'), ('arcs', 'only'), ('right', 'arcs')))
    test_dependencies("minibatch_parse", deps[1],
                      (('ROOT', 'right'), ('arcs', 'only'), ('only', 'again'), ('right', 'arcs')))
    test_dependencies("minibatch_parse", deps[2],
                      (('only', 'ROOT'), ('only', 'arcs'), ('only', 'left')))
    test_dependencies("minibatch_parse", deps[3],
                      (('again', 'ROOT'), ('again', 'arcs'), ('again', 'left'), ('again', 'only')))

    # Out-of-bound test
    sentences = [["right"]]
    deps = minibatch_parse(sentences, DummyModel(), 2)
    test_dependencies("minibatch_parse", deps[0], (('ROOT', 'right'),))

    # Mixed arcs test
    sentences = [["this", "is", "interleaving", "dependency", "test"]]
    deps = minibatch_parse(sentences, DummyModel(mode="interleave"), 1)
    test_dependencies("minibatch_parse", deps[0],
                      (('ROOT', 'is'), ('dependency', 'interleaving'),
                      ('dependency', 'test'), ('is', 'dependency'), ('is', 'this')))
    print("minibatch_parse test passed!")


# if __name__ == '__main__':
#     args = sys.argv
#     if len(args) != 2:
#         raise Exception("You did not provide a valid keyword. Either provide 'part_c' or 'part_d', when executing this script")
#     elif args[1] == "part_c":
#         test_parse_step()
#         test_parse()
#     elif args[1] == "part_d":
#         test_minibatch_parse()
#     else:
#         raise Exception("You did not provide a valid keyword. Either provide 'part_c' or 'part_d', when executing this script")

embeddings = np.zeros((100, 30), dtype=np.float32)
model = ParserModel(embeddings)

def check_embedding():
    inds = torch.randint(0, 100, (4, 36), dtype=torch.long)
    selected = model.embedding_lookup(inds)
    assert np.all(selected.data.numpy() == 0), "The result of embedding lookup: " \
                                    + repr(selected) + " contains non-zero elements."

def check_forward():
    inputs = torch.randint(0, 100, (4, 36), dtype=torch.long)
    out = model(inputs)
    expected_out_shape = (4, 3)
    assert out.shape == expected_out_shape, "The result shape of forward is: " + repr(out.shape) + \
                                            " which doesn't match expected " + repr(expected_out_shape)


In [183]:
test_parse_step()
test_parse()

SHIFT test passed!
LEFT-ARC test passed!
RIGHT-ARC test passed!
parse test passed!


In [184]:
check_embedding()

In [185]:
check_forward()

In [193]:
test_minibatch_parse()

minibatch_parse test passed!
